<a href="https://colab.research.google.com/github/emmenets/neuralnetwork/blob/master/190725Einfuehrung_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data



In [0]:
!wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

In [15]:
!tar -xvf /content/cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [0]:
DATA_DIR = '/tmp/data'
NUM_STEPS = 10000
MINIBATCH_SIZE = 100

In [0]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def conv_layer(input, shape):
  W = weight_variable(shape)
  b = bias_variable([shape[3]])
  return tf.nn.relu(conv2d(input, W) +b)

def full_layer(input, size):
  in_size = int(input.get_shape()[1])
  W = weight_variable([in_size, size])
  b = bias_variable([size])
  return tf.matmul(input, W) + b

In [0]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf. reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)

full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)

In [10]:
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for i in range(NUM_STEPS):  
    batch = mnist.train.next_batch(50)  
    
    if i % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0})
      print("Schritt {}, Genauigkeit Anlernen {}".format(i, train_accuracy))
      
    sess.run(train_step, feed_dict={x: batch[0], y_:batch[1], keep_prob: 0.5})
    
  X = mnist.test.images.reshape(10, 1000, 784)
  Y = mnist.test.labels.reshape(10, 1000, 10)
  test_accuracy = np.mean([sess.run(accuracy, feed_dict={x:X[i], y_:Y[i], keep_prob:1.0})for i in range(10)])
  
  print("Genauigkeit Test: {}".format(test_accuracy))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Schritt 0, Genauigkeit Anlernen 0.07999999821186066
Schritt 100, Genauigkeit Anlernen 0.8199999928474426
Schritt 200, Genauigkeit Anlernen 0.8999999761581421
Schritt 300, Genauigkeit Anlernen 0.8799999952316284
Schritt 400, Genauigkeit Anlernen 0.9399999976158142
Schritt 500, Genauigkeit Anlernen 0.9599999785423279
Schritt 600, Genauigkeit Anlernen 0.9399999976158142
Schritt 700, Genauigkeit Anlernen 0.8799999952316284
Schritt 800, Genauigkeit Anlernen 0.9599999785423279
Schritt 900, Genauigkeit Anlernen 0.9399999976158142
Schritt 1000, Genauigkeit Anlernen 0.9800000190734863
Schritt 1100, Genauigkeit Anlernen 0.9599999785423279
Schritt 1200, Genauigkeit Anlernen 0.8999999761581421
Schritt 1300, Genauigkeit Anlernen 0.9800000190734863
Schritt 1400, Genauigkeit Anlernen 0.9800000190734863
Schritt 150